<a href="https://colab.research.google.com/github/nurulqa/coba-berhasil/blob/main/Spatial_Quadratic_SMA_Jakarta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas

In [ ]:
!pip install pysal

In [ ]:
!pip install contextily

In [ ]:
import numpy
import pandas
import geopandas
import pysal
import seaborn
import contextily
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

In [ ]:
db = pandas.read_csv("/content/data-lat-long-sma-negeri-jkt.csv")

In [ ]:
db.info()

In [ ]:
seaborn.jointplot(x="longitude", y="latitude", data=db, s=0.5);

In [ ]:
joint_axes = seaborn.jointplot(
    x="longitude", y="latitude", data=db, s=0.5
)
contextily.add_basemap(
    joint_axes.ax_joint,
    crs="EPSG:4326",
    source=contextily.providers.CartoDB.PositronNoLabels,
);

In [ ]:
# Set up figure and axis
f, ax = plt.subplots(1, figsize=(12, 9))
# Generate and add hexbin with 50 hexagons in each
# dimension, no borderlines, half transparency,
# and the reverse viridis colormap
hb = ax.hexbin(
    db["longitude"],
    db["latitude"],
    gridsize=50,
    linewidths=0,
    alpha=0.5,
    cmap="viridis_r",
)
# Add basemap
contextily.add_basemap(
    ax, source=contextily.providers.CartoDB.Positron
)
# Add colorbar
plt.colorbar(hb)
# Remove axes
ax.set_axis_off()

In [ ]:
from pointpats import (
    distance_statistics,
    QStatistic,
    random,
    PointPattern,
)

In [ ]:
coordinates = db[["longitude","latitude"]].values
qstat = QStatistic(coordinates)
qstat.plot()

In [ ]:
qstat.chi2_pvalue

In [ ]:
random_pattern = random.poisson(coordinates, size=len(coordinates))
qstat_null = QStatistic(random_pattern)
qstat_null.plot()
qstat_null.chi2_pvalue

In [ ]:
import libpysal

alpha_shape, alpha, circs = libpysal.cg.alpha_shape_auto(
    coordinates, return_circles=True
)
random_pattern_ashape = random.poisson(
    alpha_shape, size=len(coordinates)
)
qstat_null_ashape = QStatistic(random_pattern_ashape)
qstat_null_ashape.plot()
qstat_null_ashape.chi2_pvalue

In [ ]:
import folium

loc_center = [db['latitude'].mean(), db['longitude'].mean()]

map1 = folium.Map(location = loc_center, tiles='Openstreetmap', zoom_start = 5, control_scale=True)
for index, loc in db.iterrows():
    folium.CircleMarker([loc['latitude'], loc['longitude']],     radius=2, weight=5, popup=loc['nama_sekolah']).add_to(map1)
folium.LayerControl().add_to(map1)
map1